In [38]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [39]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [40]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Experiment 1: Train Logistic Regression Classifier

In [41]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [42]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



### Experiment 3: Train XGBoost

In [43]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [44]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [45]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



<h2 align="center" style="color:blue">Track Experiments Using MLFlow</h2>

In [46]:
# List to hold different models, their configurations, and data
models = [
    ("Logistic Regression", 
      LogisticRegression(C=1, solver='liblinear'), 
      (X_train, y_train), 
      (X_test, y_test)
    ),

    ("Random Forest",
      RandomForestClassifier(n_estimators=30, max_depth=3), 
      (X_train, y_train), 
      (X_test, y_test)),

    ("XGBClassifier", 
     XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
     (X_train, y_train), 
     (X_test, y_test)
    ),
    
    ("XGBClassifier With SMOTE", 
     XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
     (X_train_res, y_train_res), 
     (X_test, y_test)
    )
]

In [47]:
# List to store classification reports for each model
reports = []

# Loop over each model
for model_name, model, train_set, test_set in models:
    X_train, y_train = train_set  # Unpack training data
    X_test, y_test = test_set  # Unpack test data

    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Predict on test data

    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)  # Store the report

### MLFlow


In [48]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [49]:
# Set the experiment name in MLFlow
mlflow.set_experiment("Anomaly-Detection") # All runs will be grouped under this experiment

# Set the MLflow tracking uri ro the local MLflow server
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

2024/10/15 14:40:43 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly-Detection' does not exist. Creating a new experiment.


In [50]:
# Log models and their performance to MLflow
for i, element in enumerate(models):
    model_name = element[0]  # Get model name
    model = element[1]  # Get model instance
    report = reports[i]  # Get classification report

    # Start an MLflow run for each model
    with mlflow.start_run(run_name=model_name):
        # Log hyperparameters (for scikit-learn models)
        mlflow.log_params(model.get_params())

        # Log key metrics from the classification report
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])

        # Log the model to MLflow
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2024/10/15 14:40:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/15 14:40:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/723941634219526615/runs/aecb68c812464ddd846dc4d42ba2d863.
2024/10/15 14:40:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/723941634219526615.
2024/10/15 14:40:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/15 14:40:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/723941634219526615/runs/4df8d9a0230c45e8a5479447e9a30d15.
2024/10/15 14:40:52 INFO mlflow.tracking._tracking_service.client: 🧪 View exp

### Register the Model


In [55]:
# Register the model with MLflow
model_name = "XGB-Smote"
run_id = input("Enter Run ID: ")  # Prompt for Run ID
model_uri = f"runs:/{run_id}/model" # model URI for registration

# Register the model in MLflow model registry
result = mlflow.register_model(
    model_uri, model_name
)

print(f"Successfully registered model '{model_name}'.")

Successfully registered model 'XGB-Smote'.
2024/10/15 14:45:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


Successfully registered model 'XGB-Smote'.


Created version '1' of model 'XGB-Smote'.


### Loading the model


In [56]:
# Load the registered model by its name and version
model_name = "XGB-Smote"
model_version = 1  # Specify model version
model_uri = f"models:/{model_name}/{model_version}"  # URI for loading the model

# Load the XGBoost model from MLflow registry
load_model = mlflow.xgboost.load_model(model_uri)

# Make predictions using the loaded model
y_pred = load_model.predict(X_test)

# Display first 4 predictions
print(y_pred[:4])

[0 0 0 0]


### Dev to Prod Environment

In [61]:
# Create the source model URI for the development model
dev_model_uri=f"models:/{model_name}/{model_version}"

prod_model="Anomaly-Detection-Prod"

# Initialize the MLflow client
client=mlflow.MlflowClient()

# Copy the model version
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)

Registered model 'Anomaly-Detection-Prod' already exists. Creating a new version of this model...
Copied version '1' of model 'XGB-Smote' to version '2' of model 'Anomaly-Detection-Prod'.


<ModelVersion: aliases=[], creation_timestamp=1728993560962, current_stage='None', description='', last_updated_timestamp=1728993560962, name='Anomaly-Detection-Prod', run_id='37b55e88a26d4f5dac3bb68869b005ab', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='2'>

In [66]:
# Load the registered production model by its name and version
model_version = 2  # Specify model version
model_uri = f"models:/{prod_model}/{model_version}"  # URI for loading the model

# Load the Production model from MLflow registry
load_model = mlflow.xgboost.load_model(model_uri)

# Make predictions using the loaded model
y_pred = load_model.predict(X_test)

# Display first 4 predictions
print(y_pred[:4])

[0 0 0 0]
